# Time-varying bid problems with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Sourabh Dalvi

## Introduction

PowerSimulations.jl supports the construction of Operations problems in power system
with three part cost bids for each time step. MarketBidCost allows the user to pass a
time-series of variable cost for energy and ancillary services jointly.
This example shows how to build a Operations problem with MarketBidCost and how to add
the time-series data to the devices.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using D3TypeTrees

### Data management packages

In [3]:
using Dates
using DataFrames
using TimeSeries

### Optimization packages

In [4]:
using Cbc #solver

### Data
This data depends upon the [RTS-GMLC](https://github.com/gridmod/rts-gmlc) dataset. Let's
download and extract the data.

In [5]:
rts_dir = SIIPExamples.download("https://github.com/GridMod/RTS-GMLC")
rts_src_dir = joinpath(rts_dir, "RTS_Data", "SourceData")
rts_siip_dir = joinpath(rts_dir, "RTS_Data", "FormattedData", "SIIP");

### Create a `System` from RTS-GMLC data just like we did in the [parsing tabular data example.](../../notebook/2_PowerSystems_examples/04_parse_tabulardata.jl)

In [6]:
rawsys = PowerSystems.PowerSystemTableData(
    rts_src_dir,
    100.0,
    joinpath(rts_siip_dir, "user_descriptors.yaml"),
    timeseries_metadata_file = joinpath(rts_siip_dir, "timeseries_pointers.json"),
    generator_mapping_file = joinpath(rts_siip_dir, "generator_mapping.yaml"),
);
sys = System(rawsys; time_series_resolution = Dates.Hour(1));

┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed bus.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in dc_branch.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed dc_branch.csv
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in gen.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages

┌ Info: Adding contributing generators for Flex_Up by category
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:675
┌ Warning: User-defined column name Startup Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Shutdown Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Status at Start is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Time at Status is not in dataframe.
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/power_system_table_data.jl:1363
┌ Warning: User-defined column name Start Cost Cold is not in datafr

### Creating the Time Series data for Energy bid

In [7]:
MultiDay = collect(
    DateTime("2020-01-01T00:00:00"):Hour(1):(DateTime("2020-01-01T00:00:00") + Hour(8783)),
);

### Replacing existing ThreePartCost with MarketBidCost
Here we replace the existing ThreePartCost with MarketBidCost, and add the energy bid
time series to the system. The TimeSeriesData that holds the energy bid data can be of any
type (i.e. `SingleTimeSeries` or `Deterministic`) and bid data should be of type
`Array{Float64}`,`Array{Tuple{Float64, Float64}}` or `Array{Array{Tuple{Float64,Float64}}`.

In [8]:
for gen in get_components(ThermalGen, sys)
    varcost = get_operation_cost(gen)
    market_bid_cost = MarketBidCost(;
        variable = nothing,
        no_load = get_fixed(varcost),
        start_up = (hot = get_start_up(varcost), warm = 0.0, cold = 0.0),
        shut_down = get_shut_down(varcost),
        ancillary_services = Vector{Service}(),
    )
    set_operation_cost!(gen, market_bid_cost)

    data = TimeArray(MultiDay, repeat([get_cost(get_variable(varcost))], 8784))
    _time_series = SingleTimeSeries("variable_cost", data)
    set_variable_cost!(sys, gen, _time_series)
end

### Transforming SingleTimeSeries into Deterministic

In [9]:
horizon = 24;
interval = Dates.Hour(24);
transform_single_time_series!(sys, horizon, interval)

In the [OperationsProblem example](../../notebook/3_PowerSimulations_examples/1_operations_problems.ipynb)
we defined a unit-commitment problem with a copper plate representation of the network.
Here, we want do define unit-commitment problem  with ThermalMultiStartUnitCommitment
formulation for thermal device representation.

For now, let's just choose a standard ACOPF formulation.

In [10]:
uc_template = template_unit_commitment(network = CopperPlatePowerModel)


Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = HydroDispatchRunOfRiver
      Generators:
        device_type = ThermalStandard
        formulation = ThermalBasicUnitCommitment
      DistRE:
        device_type = RenewableFix
        formulation = FixedOutput
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = StaticL

Currently  energy budget data isn't stored in the RTS-GMLC dataset.

In [11]:
uc_template.devices[:Generators] =
    DeviceModel(ThermalStandard, ThermalMultiStartUnitCommitment)

solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

Now we can build a 4-hour economic dispatch / ACOPF problem with the RTS data.

In [12]:
problem = OperationsProblem(
    EconomicDispatchProblem,
    uc_template,
    sys,
    horizon = 4,
    optimizer = solver,
    balance_slack_variables = true,
)

┌ Info: Unit System changed to InfrastructureSystems.UnitSystemModule.UnitSystem.SYSTEM_BASE = 0
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/base.jl:289
┌ Warning: The data doesn't include devices of type InterruptibleLoad, consider changing the device models
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/devices_models/device_constructors/common/constructor_validations.jl:3
┌ Warning: No variable cost defined for 201_HYDRO_4
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/devices_models/devices/common/cost_functions.jl:346
┌ Warning: Data doesn't contain generators with ramp limits, consider adjusting your formulation
└ @ PowerSimulations /Users/cbarrows/.julia/packages/PowerSimulations/If05p/src/devices_models/devices/thermal_generation.jl:497



Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      ILoads:
        device_type = InterruptibleLoad
        formulation = InterruptiblePowerLoad
      HydroROR:
        device_type = HydroDispatch
        formulation = HydroDispatchRunOfRiver
      Generators:
        device_type = ThermalStandard
        formulation = ThermalMultiStartUnitCommitment
      DistRE:
        device_type = RenewableFix
        formulation = FixedOutput
      Hydro:
        device_type = HydroEnergyReservoir
        formulation = HydroDispatchRunOfRiver
      Loads:
        device_type = PowerLoad
        formulation = StaticPowerLoad
      RE:
        device_type = RenewableDispatch
        formulation = RenewableFullDispatch
  branches: 
      T:
        device_type = Transformer2W
        formulation = StaticTransformer
      TT:
        device_type = TapTransformer
        formulation = StaticTransformer
      L:
        device_type = Line
        formulation = St

And solve it ...

In [13]:
solve!(problem)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 61235.2 - 0.02 seconds
Cgl0003I 142 fixed, 0 tightened bounds, 210 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 149 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 80 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 44 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 16 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0004I processed model has 1413 rows, 3500 columns (612 integer (612 of which binary)) and 7788 elements
Cbc0045I Trying just fixing integer variables (and fixingish SOS).
Cbc0045I MIPStart provided solution wi

PowerSimulations.OperationsProblemResults(100.0, Dict{Symbol,DataFrame}(:P__ThermalStandard => 4×76 DataFrame
 Row │ 322_CT_6  321_CC_1  202_STEAM_3  315_STEAM_1  223_CT_4  223_CT_6  313_C ⋯
     │ Float64   Float64   Float64      Float64      Float64   Float64   Float ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      0.0       0.0          0.0          0.0       0.0       0.0        ⋯
   2 │      0.0       0.0          0.0          0.0       0.0       0.0
   3 │      0.0       0.0          0.0          0.0       0.0       0.0
   4 │      0.0       0.0          0.0          0.0       0.0       0.0
                                                              70 columns omitted,:P__RenewableDispatch => 4×30 DataFrame
 Row │ 122_WIND_1  324_PV_3  312_PV_1  102_PV_1  101_PV_1  324_PV_2  313_PV_2  ⋯
     │ Float64     Float64   Float64   Float64   Float64   Float64   Float64   ⋯
─────┼─────────────────────────────────────────────────────────────

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*